In [1]:
import pandas as pd
import altair as alt
import numpy as np
import openpyxl

In [2]:

nhex_total = pd.read_csv('https://raw.githubusercontent.com/RibbitsM/368-research-proj/refs/heads/main/Data/nhex_prov_terr_data.csv',
                        header=4, skiprows=lambda x: x > 51, dtype=np.float64, thousands=',', na_values='—')
nhex_percapita = pd.read_csv('https://raw.githubusercontent.com/RibbitsM/368-research-proj/refs/heads/main/Data/nhex_prov_terr_data.csv',
                        header=112, skiprows=lambda x: x > 160, dtype=np.float64, thousands=',', na_values='—')

In [3]:
nhex_total.head()

,Year,N.L.,P.E.I.,N.S.,N.B.,Que.,Ont.,Man.,Sask.,Alta.,B.C.,Y.T.,N.W.T,Nun.,Canada
0,1975.0,199.2,41.5,266.9,203.7,2531.2,3143.5,376.7,302.1,694.9,928.2,6.2,15.2,NaN,8709.3
1,1976.0,219.0,45.4,302.7,242.4,2972.5,3612.5,449.3,364.1,811.6,1082.7,8.8,19.0,NaN,10129.9
2,1977.0,233.5,50.3,334.6,271.7,3269.2,3929.1,497.3,410.8,879.1,1193.1,10.0,23.4,NaN,11102.0
3,1978.0,258.3,56.4,370.0,299.1,3661.8,4229.4,524.9,445.8,1020.5,1363.8,11.4,28.1,NaN,12269.4
4,1979.0,297.3,62.5,413.1,336.7,4088.1,4565.8,569.7,508.5,1258.2,1556.5,12.4,27.9,NaN,13696.6


In [4]:
nhex_percapita.columns = ['Year', 'N.L.', 'P.E.I.', 'N.S.', 'N.B.', 'Que.', 'Ont.', 'Man.', 'Sask.', 'Alta.', 
                        'B.C.', 'Y.T.', 'N.W.T', 'Nun.', 'Canada (Average)']
nhex_percapita.head()

,Year,N.L.,P.E.I.,N.S.,N.B.,Que.,Ont.,Man.,Sask.,Alta.,B.C.,Y.T.,N.W.T,Nun.,Canada (Average)
0,1975.0,357.92,352.59,322.88,300.87,399.86,377.83,367.52,329.31,384.21,371.34,281.70,355.89,NaN,376.32
1,1976.0,389.23,382.68,362.39,351.60,464.68,429.35,435.49,390.82,434.15,427.29,393.11,427.60,NaN,431.98
2,1977.0,412.98,419.50,398.29,390.49,508.18,462.03,479.36,434.85,451.21,464.19,444.82,525.96,NaN,467.93
3,1978.0,455.02,463.22,438.04,427.56,568.57,492.35,504.31,468.12,504.61,521.51,492.42,619.82,NaN,512.01
4,1979.0,521.43,508.78,486.34,478.80,632.25,527.10,549.26,529.79,600.00,583.99,538.34,610.04,NaN,565.94


In [5]:
keep_attributes = ['geogprv','PHC_060','PHC_035','PHC_005','PHC_020','UCN_005']
cchs_data = pd.read_stata("Data\cchs_201920_pumf.dta", columns=keep_attributes,convert_categoricals=False)
cchs_data.shape

(108252, 6)

In [6]:
for name in cchs_data['geogprv'].unique():
    print(name)

47
59
13
46
35
12
24
10
48
11
60


In [7]:
prov_dict = {47:"Sask.",59:'B.C.',13:'N.B.',46:'Man.',35:'Ont.',12:'N.S.',24:'Que.',10:'N.L.', 
            48: 'Alta.', 11: 'P.E.I.', 60: 'Terrs.'}
cchs_data['geogprv'] = cchs_data['geogprv'].replace(prov_dict)

cchs_data_avg = cchs_data.groupby('geogprv',axis=0).mean().reset_index()
cchs_data_avg

C:\Users\rowan\AppData\Local\Temp\ipykernel_16580\2323738982.py:5: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  cchs_data_avg = cchs_data.groupby('geogprv',axis=0).mean().reset_index()


,geogprv,PHC_060,PHC_035,PHC_005,PHC_020,UCN_005
0,Alta.,2.682424,3.004131,1.095805,1.127307,NaN
1,B.C.,2.831505,3.243095,1.105276,1.151428,1.944629
2,Man.,2.598450,3.239828,1.102946,1.112283,1.965898
3,N.B.,2.316149,3.931027,1.073584,1.071775,1.962131
4,N.L.,2.292573,3.606378,1.051239,1.104966,1.964641
5,N.S.,2.499811,4.010989,1.092441,1.113047,1.935490
6,Ont.,2.649874,3.163832,1.072835,1.087139,1.951512
7,P.E.I.,2.271487,3.336353,1.071032,1.089985,NaN
8,Que.,2.771138,3.808955,1.150822,1.170200,1.953313
9,Sask.,2.542336,2.977078,1.093136,1.125060,NaN


In [14]:
survey_year_expenditure = nhex_percapita[44:46]
survey_year_expenditure['Terrs'] = ((survey_year_expenditure['Y.T.'] + survey_year_expenditure['N.W.T'] +
                                    survey_year_expenditure['Nun.'])/3)
survey_year_expenditure = survey_year_expenditure.drop(['Year', 'Canada (Average)','Y.T.','N.W.T','Nun.'], axis=1)
nhex_pivot = survey_year_expenditure.melt(var_name='geogprv',value_name='avg_percap_spend'
                                        ).groupby('geogprv').mean().reset_index()
nhex_pivot

C:\Users\rowan\AppData\Local\Temp\ipykernel_16580\391942428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_year_expenditure['Terrs'] = ((survey_year_expenditure['Y.T.'] + survey_year_expenditure['N.W.T'] +


,geogprv,avg_percap_spend
0,Alta.,5317.510
1,B.C.,4762.885
2,Man.,5021.065
3,N.B.,4593.585
4,N.L.,6319.605
5,N.S.,5121.230
6,Ont.,4610.650
7,P.E.I.,5101.495
8,Que.,5037.435
9,Sask.,5198.730


In [15]:
combined_data = nhex_pivot.merge(cchs_data_avg, on='geogprv')
combined_data

,geogprv,avg_percap_spend,PHC_060,PHC_035,PHC_005,PHC_020,UCN_005
0,Alta.,5317.510,2.682424,3.004131,1.095805,1.127307,NaN
1,B.C.,4762.885,2.831505,3.243095,1.105276,1.151428,1.944629
2,Man.,5021.065,2.598450,3.239828,1.102946,1.112283,1.965898
3,N.B.,4593.585,2.316149,3.931027,1.073584,1.071775,1.962131
4,N.L.,6319.605,2.292573,3.606378,1.051239,1.104966,1.964641
5,N.S.,5121.230,2.499811,4.010989,1.092441,1.113047,1.935490
6,Ont.,4610.650,2.649874,3.163832,1.072835,1.087139,1.951512
7,P.E.I.,5101.495,2.271487,3.336353,1.071032,1.089985,NaN
8,Que.,5037.435,2.771138,3.808955,1.150822,1.170200,1.953313
9,Sask.,5198.730,2.542336,2.977078,1.093136,1.125060,NaN


In [32]:
PHC_060_chart = alt.Chart(combined_data).mark_circle().encode(
    alt.X('PHC_060').scale(domain=[2,3]),
    alt.Y('avg_percap_spend').scale(domain=[4000, 6500]),
    alt.Color('geogprv'))

PHC_035_chart = alt.Chart(combined_data).mark_circle().encode(
    alt.X('PHC_035').scale(domain=[2.75,4.25]),
    alt.Y('avg_percap_spend').scale(domain=[4000, 6500]),
    alt.Color('geogprv'))

PHC_005_chart = alt.Chart(combined_data).mark_circle().encode(
    alt.X('PHC_005').scale(domain=[1,1.25]),
    alt.Y('avg_percap_spend').scale(domain=[4000, 6500]),
    alt.Color('geogprv'))

PHC_020_chart = alt.Chart(combined_data).mark_circle().encode(
    alt.X('PHC_020').scale(domain=[1,1.25]),
    alt.Y('avg_percap_spend').scale(domain=[4000, 6500]),
    alt.Color('geogprv'))

UCN_005_chart = alt.Chart(combined_data).mark_circle().encode(
    alt.X('UCN_005').scale(domain=[1.9,2.1]),
    alt.Y('avg_percap_spend').scale(domain=[4000, 6500]),
    alt.Color('geogprv'))

PHC_060_chart & PHC_020_chart | PHC_035_chart & UCN_005_chart | PHC_005_chart 

alt.HConcatChart(...)